In [1]:
import numpy

class StepPattern:

    def __init__(self, mx, hint="NA"):
        self.mx = numpy.array(mx, dtype=numpy.double)
        self.hint = hint

    def get_n_rows(self):
        """Total number of steps in the recursion."""
        return self.mx.shape[0]

    def get_n_patterns(self):
        """Number of rules in the recursion."""
        return int(numpy.max(self.mx[:, 0]))

    def T(self):
        """Transpose a step pattern."""
        tsp = self
        tsp.mx = tsp.mx[:, [0, 2, 1, 3]]
        if tsp.hint == "N":
            tsp.hint = "M"
        elif tsp.hint == "M":
            tsp.hint = "N"
        return tsp

    def __str__(self):
        np = self.get_n_patterns()
        head = " g[i,j] = min(\n"

        body = ""
        for p in range(1, np + 1):
            steps = self._extractpattern(p)
            ns = steps.shape[0]
            steps = numpy.flip(steps, 0)

            for s in range(ns):
                di, dj, cc = steps[s, :]
                dis = "" if di == 0 else f"{-int(di)}"
                djs = "" if dj == 0 else f"{-int(dj)}"
                dijs = f"i{dis:2},j{djs:2}"

                if cc == -1:
                    gs = f"    g[{dijs}]"
                    body = body + " " + gs
                else:
                    ccs = "    " if cc == 1 else f"{cc:2.2g} *"
                    ds = f"+{ccs} d[{dijs}]"
                    body = body + " " + ds
            body = body + " ,\n"

        tail = " ) \n\n"
        ntxt = f"Normalization hint: {self.hint}\n"

        return "Step pattern recursion:\n" + head + body + tail + ntxt

    def plot(self):
        """Provide a visual description of a StepPattern object"""
        import matplotlib.pyplot as plt
        x = self.mx
        pats = numpy.arange(1, 1 + self.get_n_patterns())

        alpha = .5
        fudge = [0, 0]

        fig, ax = plt.subplots(figsize=(6, 6))
        for i in pats:
            ss = x[:, 0] == i
            ax.plot(-x[ss, 1], -x[ss, 2], lw=2, color="tab:blue")
            ax.plot(-x[ss, 1], -x[ss, 2], 'o', color="black", marker="o", fillstyle="none")

            if numpy.sum(ss) == 1: continue

            xss = x[ss, :]
            xh = alpha * xss[:-1, 1] + (1 - alpha) * xss[1:, 1]
            yh = alpha * xss[:-1, 2] + (1 - alpha) * xss[1:, 2]

            for xx, yy, tt in zip(xh, yh, xss[1:, 3]):
                ax.annotate("{:.2g}".format(tt), (-xx - fudge[0],
                                                  -yy - fudge[1]))

        endpts = x[:, 3] == -1
        ax.plot(-x[endpts, 1], -x[endpts, 2], 'o', color="black")

        ax.set_xlabel("Query index")
        ax.set_ylabel("Reference index")
        ax.set_xticks(numpy.unique(-x[:, 1]))
        ax.set_yticks(numpy.unique(-x[:, 2]))
        plt.show()
        return ax

    def _extractpattern(self, sn):
        sp = self.mx
        sbs = sp[:, 0] == sn
        spl = sp[sbs, 1:]
        return numpy.flip(spl, 0)

    def _mkDirDeltas(self):
        out = numpy.array(self.mx, dtype=numpy.int32)
        out = out[out[:, 3] == -1, :]
        out = out[:, [1, 2]]
        return out

    def _get_p(self):
        # Dimensions are reversed wrt R
        s = self.mx[:, [0, 2, 1, 3]]
        return s.T.reshape(-1)



# Alternate constructor for ease of R import
def _c(*v):
    va = numpy.array([*v])
    if len(va) % 4 != 0:
        _error("Internal error in _c constructor")
    va = va.reshape((-1, 4))
    return (va)


# Kludge because lambda: raise doesn't work
def _error(s):
    raise ValueError(s)


##################################################
##################################################

# Reimplementation of the building process

class _P:
    def __init__(self, pid, subtype, smoothing):
        self.subtype = subtype
        self.smoothing = smoothing
        self.pid = pid
        self.i = [0]
        self.j = [0]

    def step(self, di, dj):  # equivalent to .Pstep
        self.i.append(di)
        self.j.append(dj)
        return self

    def get(self):  # eqv to .Pend
        ia = numpy.array(self.i, dtype=numpy.double)
        ja = numpy.array(self.j, dtype=numpy.double)
        si = numpy.cumsum(ia)
        sj = numpy.cumsum(ja)
        ni = numpy.max(si) - si  # ?
        nj = numpy.max(sj) - sj
        if self.subtype == "a":
            w = numpy.minimum(ia, ja)
        elif self.subtype == "b":
            w = numpy.maximum(ia, ja)
        elif self.subtype == "c":
            w = ia
        elif self.subtype == "d":
            w = ia + ja
        else:
            _error("Unsupported subtype")

        if self.smoothing:
            # if self.pid==3:                import ipdb; ipdb.set_trace()
            w[1:] = numpy.mean(w[1:])

        w[0] = -1.0

        nr = len(w)
        mx = numpy.zeros((nr, 4))
        mx[:, 0] = self.pid
        mx[:, 1] = ni
        mx[:, 2] = nj
        mx[:, 3] = w
        return mx


## Row P=2
symmetricP2 = StepPattern(_c(
    1, 2, 3, -1,
    1, 1, 2, 2,
    1, 0, 1, 2,
    1, 0, 0, 1,
    2, 1, 1, -1,
    2, 0, 0, 2,
    3, 3, 2, -1,
    3, 2, 1, 2,
    3, 1, 0, 2,
    3, 0, 0, 1
), "N+M");

symmetricP3 = StepPattern(_c(
    1, 3, 4, -1,
    1, 2, 3, 2,
    1, 1, 2, 2,
    1, 0, 1, 2,
    1, 0, 0, 1,
    2, 1, 1, -1,
    2, 0, 0, 2,
    3, 4, 3, -1,
    3, 3, 2, 2,
    3, 2, 1, 2,
    3, 1, 0, 2,
    3, 0, 0, 1
), "N+M");

symmetricP4 = StepPattern(_c(
    1, 4, 5, -1,
    1, 3, 4, 2,
    1, 2, 3, 2,
    1, 1, 2, 2,
    1, 0, 1, 2,
    1, 0, 0, 1,
    2, 1, 1, -1,
    2, 0, 0, 2,
    3, 5, 4, -1,
    3, 4, 3, 2,
    3, 3, 2, 2,
    3, 2, 1, 2,
    3, 1, 0, 2,
    3, 0, 0, 1
), "N+M");

symmetricP5 = StepPattern(_c(
    1, 5, 6, -1,
    1, 4, 5, 2,
    1, 3, 4, 2,
    1, 2, 3, 2,
    1, 1, 2, 2,
    1, 0, 1, 2,
    1, 0, 0, 1,
    2, 1, 1, -1,
    2, 0, 0, 2,
    3, 6, 5, -1,
    3, 5, 4, 2,
    3, 4, 3, 2,
    3, 3, 2, 2,
    3, 2, 1, 2,
    3, 1, 0, 2,
    3, 0, 0, 1
), "N+M");

symmetricP6 = StepPattern(_c(
    1, 6, 7, -1,
    1, 5, 6, 2,
    1, 4, 5, 2,
    1, 3, 4, 2,
    1, 2, 3, 2,
    1, 1, 2, 2,
    1, 0, 1, 2,
    1, 0, 0, 1,
    2, 1, 1, -1,
    2, 0, 0, 2,
    3, 7, 6, -1,
    3, 6, 5, 2,
    3, 5, 4, 2,
    3, 4, 3, 2,
    3, 3, 2, 2,
    3, 2, 1, 2,
    3, 1, 0, 2,
    3, 0, 0, 1
), "N+M");

symmetricP7 = StepPattern(_c(
    1, 7, 8, -1,
    1, 6, 7, 2,
    1, 5, 6, 2,
    1, 4, 5, 2,
    1, 3, 4, 2,
    1, 2, 3, 2,
    1, 1, 2, 2,
    1, 0, 1, 2,
    1, 0, 0, 1,
    2, 1, 1, -1,
    2, 0, 0, 2,
    3, 8, 7, -1,
    3, 7, 6, 2,
    3, 6, 5, 2,
    3, 5, 4, 2,
    3, 4, 3, 2,
    3, 3, 2, 2,
    3, 2, 1, 2,
    3, 1, 0, 2,
    3, 0, 0, 1
), "N+M");

symmetricP8 = StepPattern(_c(
    1, 8, 9, -1,
    1, 7, 8, 2,
    1, 6, 7, 2,
    1, 5, 6, 2,
    1, 4, 5, 2,
    1, 3, 4, 2,
    1, 2, 3, 2,
    1, 1, 2, 2,
    1, 0, 1, 2,
    1, 0, 0, 1,
    2, 1, 1, -1,
    2, 0, 0, 2,
    3, 9, 8, -1,
    3, 8, 7, 2,
    3, 7, 6, 2,
    3, 6, 5, 2,
    3, 5, 4, 2,
    3, 4, 3, 2,
    3, 3, 2, 2,
    3, 2, 1, 2,
    3, 1, 0, 2,
    3, 0, 0, 1
), "N+M");

symmetricP9 = StepPattern(_c(
    1, 9, 10, -1,
    1, 8, 9, 2,
    1, 7, 8, 2,
    1, 6, 7, 2,
    1, 5, 6, 2,
    1, 4, 5, 2,
    1, 3, 4, 2,
    1, 2, 3, 2,
    1, 1, 2, 2,
    1, 0, 1, 2,
    1, 0, 0, 1,
    2, 1, 1, -1,
    2, 0, 0, 2,
    3, 10, 9, -1,
    3, 9, 8, 2,
    3, 8, 7, 2,
    3, 7, 6, 2,
    3, 6, 5, 2,
    3, 5, 4, 2,
    3, 4, 3, 2,
    3, 3, 2, 2,
    3, 2, 1, 2,
    3, 1, 0, 2,
    3, 0, 0, 1
), "N+M");

symmetricP10 = StepPattern(_c(
    1, 10, 11, -1,
    1, 9, 10, 2,
    1, 8, 9, 2,
    1, 7, 8, 2,
    1, 6, 7, 2,
    1, 5, 6, 2,
    1, 4, 5, 2,
    1, 3, 4, 2,
    1, 2, 3, 2,
    1, 1, 2, 2,
    1, 0, 1, 2,
    1, 0, 0, 1,
    2, 1, 1, -1,
    2, 0, 0, 2,
    3, 11, 10, -1,
    3, 10, 9, 2,
    3, 9, 8, 2,
    3, 8, 7, 2,
    3, 7, 6, 2,
    3, 6, 5, 2,
    3, 5, 4, 2,
    3, 4, 3, 2,
    3, 3, 2, 2,
    3, 2, 1, 2,
    3, 1, 0, 2,
    3, 0, 0, 1
), "N+M");
